In [ ]:
!wget --continue --user="user" --password="password" "https://nextcloud.siwatsystem.com/public.php/webdav/" -O i-squared-hackathon-data.zip

--2025-02-15 12:15:37--  https://nextcloud.siwatsystem.com/public.php/webdav/
Resolving nextcloud.siwatsystem.com (nextcloud.siwatsystem.com)... 171.6.133.243
Connecting to nextcloud.siwatsystem.com (nextcloud.siwatsystem.com)|171.6.133.243|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Nextcloud", charset="UTF-8"
Reusing existing connection to nextcloud.siwatsystem.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 3217204537 (3.0G) [application/zip]
Saving to: ‘i-squared-hackathon-data.zip’

i-squared-hackathon  33%[=====>              ]   1.00G  10.5MB/s    in 1m 50s  

2025-02-15 12:17:28 (9.39 MB/s) - Connection closed at byte 1079627656. Retrying.

--2025-02-15 12:17:29--  (try: 2)  https://nextcloud.siwatsystem.com/public.php/webdav/
Connecting to nextcloud.siwatsystem.com (nextcloud.siwatsystem.com)|171.6.133.243|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 

In [ ]:
!unzip -qq i-squared-hackathon-data.zip

replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
!rm vit_model_augmented.pth vit_model_fast.pth yolov8s.pt test_predictions_merged.csv test_predictions_augmented.csv test_predictions_augmented_cropped.csv

rm: cannot remove 'vit_model_augmented.pth': No such file or directory
rm: cannot remove 'vit_model_fast.pth': No such file or directory
rm: cannot remove 'test_predictions_merged.csv': No such file or directory
rm: cannot remove 'test_predictions_augmented.csv': No such file or directory
rm: cannot remove 'test_predictions_augmented_cropped.csv': No such file or directory


In [ ]:
!git clone https://github.com/KheawKachee/privateforwhile.git

Cloning into 'privateforwhile'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 82.15 KiB | 672.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
!mv /content/privateforwhile/labels_summary.csv /content/labels_summary.csv

In [ ]:
!rm -rf /content/dataset/relabeled_train
!rm -rf /content/privateforwhile

In [ ]:
import pandas as pd

# Load CSV file into a DataFrame
df = pd.read_csv('labels_summary.csv')

# Filter out rows where 'label' is 'unknown' or empty (adjust condition as needed)
filtered_df = df[df['label'] != 'unknown']

# Overwrite the original CSV with the filtered data
filtered_df.to_csv('labels_summary.csv', index=False)

print("CSV file overwritten with filtered data.")


CSV file overwritten with filtered data.


In [ ]:
!pip install torch torchvision transformers scikit-learn albumentations opencv-python ultralytics

In [ ]:
import os
import csv
import shutil

# Define paths
source_dir = "/content/dataset/train"  # Root train directory
csv_file = "/content/labels_summary.csv"
destination_dir = "/content/dataset/relabeled_train"

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Load label mapping from CSV
label_map = {}
with open(csv_file, newline='') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    for row in reader:
        old_label, new_label = row[0], row[1]
        label_map[old_label] = new_label

print(f"Label map loaded with {len(label_map)} entries.")  # Progress check

# Process files
total_files = 0
moved_files = 0

for subfolder in os.listdir(source_dir):
    subfolder_path = os.path.join(source_dir, subfolder)
    if os.path.isdir(subfolder_path):
        print(f"Processing subfolder: {subfolder}")  # Progress check for each subfolder
        for file in os.listdir(subfolder_path):
            old_path = os.path.join(subfolder_path, file)
            if os.path.isfile(old_path):  # Ensure it's a file
                total_files += 1
                new_label = label_map.get(subfolder, subfolder)  # Default to same label if not in CSV
                new_folder_path = os.path.join(destination_dir, new_label)
                os.makedirs(new_folder_path, exist_ok=True)
                shutil.move(old_path, os.path.join(new_folder_path, file))
                moved_files += 1
                print(f"Moved: {file} to {new_folder_path}")  # Progress check for each moved file

print(f"✅ Relabeling complete! Total files: {total_files}, Files moved: {moved_files}")


Label map loaded with 3182 entries.
Processing subfolder: grab
Moved: cca0227024fd0585d2d3e71c70caa58a.jpeg to /content/dataset/relabeled_train/grab
Moved: 63d03c4dc03e324ec99ca16036505642.jpeg to /content/dataset/relabeled_train/grab
Moved: 90c44bb1a68c9987bc2bbbb6ea5c0e18.jpeg to /content/dataset/relabeled_train/grab
Moved: 8b27a6cabd366fd6cde3e57d88267664.jpeg to /content/dataset/relabeled_train/grab
Moved: 6ab0cae2d5e43a8ac39fa0e4be8c5d44.jpeg to /content/dataset/relabeled_train/grab
Moved: f03fd987b728f87ba0d0cdb733ad7c41.jpeg to /content/dataset/relabeled_train/grab
Moved: 412c0f785ff9dce82ac8ed22cd77c93f.jpeg to /content/dataset/relabeled_train/grab
Moved: d4e1acea0becd23f5cff91d95711d71e.jpeg to /content/dataset/relabeled_train/grab
Moved: 41cfbb772964a973cfdc24b4f28ac174.jpeg to /content/dataset/relabeled_train/grab
Moved: 386c81ab6d2a9ed0c796954ba3d459be.jpeg to /content/dataset/relabeled_train/grab
Moved: 28f99b82ade4410220fb173896e99078.jpeg to /content/dataset/relabeled_tr

In [ ]:
!rm -rf /content/dataset/cropped
!rm -rf /content/dataset/relabeled_train

In [ ]:
#yolocrop the train
import os
import cv2
import numpy as np
from sklearn.metrics import roc_auc_score
from ultralytics import YOLO

def process_image(image_path, output_path, model, y_true, y_pred):
    """Process a single image with YOLO detection"""
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Failed to load image: {image_path}")
        return

    # Run YOLO detection
    results = model(image)

    # Store detected humans & motorbikes
    humans = []
    motorbikes = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls.item())
            bbox = list(map(int, box.xyxy[0].tolist()))
            if cls == PERSON_ID:
                humans.append(bbox)
            elif cls == MOTORBIKE_ID:
                motorbikes.append(bbox)

    # For AUC calculation, we'll treat detection as binary classification
    y_true.append(1 if humans else 0)  # 1 if human detected, else 0
    y_pred.append(1 if motorbikes else 0)  # 1 if motorbike detected, else 0

    # Find the human-bike pair with the most intersection
    best_pair = None
    max_intersection = 0
    for person in humans:
        for bike in motorbikes:
            inter_area = intersection_area(person, bike, image.shape)
            if inter_area > max_intersection:
                max_intersection = inter_area
                best_pair = (person, bike)

    # Crop and save the most intersecting pair
    if best_pair:
        x1 = min(best_pair[0][0], best_pair[1][0])
        y1 = min(best_pair[0][1], best_pair[1][1])
        x2 = max(best_pair[0][2], best_pair[1][2])
        y2 = max(best_pair[0][3], best_pair[1][3])
        cropped_img = image[y1:y2, x1:x2]

        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        cv2.imwrite(output_path, cropped_img)
        print(f"✅ Saved cropped image: {output_path}")
    else:
        print(f"⚠️ No valid person-motorbike pair found in: {image_path}")

def intersection_area(boxA, boxB, image_shape):
    """Calculate the intersection area between two bounding boxes."""
    x1 = max(boxA[0], boxB[0])
    y1 = max(boxA[1], boxB[1])
    x2 = min(boxA[2], boxB[2])
    y2 = min(boxA[3], boxB[3])

    padding = 20
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(image_shape[1], x2 + padding)
    y2 = min(image_shape[0], y2 + padding)

    inter_width = max(0, x2 - x1)
    inter_height = max(0, y2 - y1)

    return inter_width * inter_height

def process_directory(input_dir, output_dir, model):
    """Recursively process all images in input directory and its subdirectories"""
    # Get the total number of images to process
    total_images = sum([len(files) for _, _, files in os.walk(input_dir)])
    processed_images = 0

    # Initialize lists for AUC score calculation
    y_true = []
    y_pred = []

    # Walk through all subdirectories
    for root, dirs, files in os.walk(input_dir):
        # Create relative path to maintain folder structure
        rel_path = os.path.relpath(root, input_dir)
        output_subdir = os.path.join(output_dir, rel_path)

        # Process each image file
        for file_name in files:
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file_name)
                output_path = os.path.join(output_subdir, file_name)

                process_image(input_path, output_path, model, y_true, y_pred)

                processed_images += 1
                print(f"Progress: {processed_images}/{total_images} images processed")

        # Compute AUC after each batch
        if processed_images > 0:
            auc = roc_auc_score(y_true, y_pred)
            print(f"Epoch AUC: {auc:.4f}")
            y_true.clear()  # Reset ground truth
            y_pred.clear()  # Reset predictions

def main():
    # Load YOLOv8 model (pretrained on COCO)
    model = YOLO("yolov8s.pt")

    # Define input & output directories
    input_dir = "/content/dataset/relabeled_train"
    output_dir = "/content/dataset/cropped"

    # COCO class IDs
    global PERSON_ID, MOTORBIKE_ID
    PERSON_ID = 0       # COCO class ID for "person"
    MOTORBIKE_ID = 3    # COCO class ID for "motorbike"

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Process all images in directory and subdirectories
    process_directory(input_dir, output_dir, model)

    print("🚀 Processing complete! All images processed.")

if __name__ == "__main__":
    main()

100%|██████████| 21.5M/21.5M [00:00<00:00, 191MB/s]


เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
✅ Saved cropped image: /content/dataset/cropped/normal/99ba3139addd795988150de3df2266c6.jpeg
Progress: 2346/3345 images processed

0: 384x640 7 persons, 1 car, 1 motorcycle, 1 handbag, 12.3ms
Speed: 4.1ms preprocess, 12.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
✅ Saved cropped image: /content/dataset/cropped/normal/2e989bbe9a189045d0de352c6cd9b330.jpeg
Progress: 2347/3345 images processed

0: 384x640 3 persons, 2 bicycles, 4 motorcycles, 12.2ms
Speed: 3.9ms preprocess, 12.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
✅ Saved cropped image: /content/dataset/cropped/normal/6f1e28b32fc231ac6da4d4d97559118a.jpeg
Progress: 2348/3345 images processed

0: 384x640 1 person, 1 motorcycle, 2 potted plants, 1 refrigerator, 12.5ms
Speed: 4.0ms preprocess, 12.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
✅ Saved cropped image: /content/dataset/cropped/normal/7d38d9388b11

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset, Subset
from transformers import ViTForImageClassification
from sklearn.metrics import roc_auc_score
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import random
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

# ----------------------------
# Configurations
# ----------------------------
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 3e-5
NUM_WORKERS = 16
PIN_MEMORY = True
K_FOLDS = 3

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

set_seed(420)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Focal Loss Definition
# ----------------------------
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, num_classes=3):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.num_classes = num_classes
        self.ce_loss = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce_loss(inputs, targets)
        pt = torch.exp(-ce_loss)  # Probabilities for correct class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

# ----------------------------
# Optimized Augmentations
# ----------------------------
def get_train_transform():
    return A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.RandomRotate90(p=0.2),
        A.HueSaturationValue(p=0.2),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        A.ElasticTransform(alpha=1, sigma=50, p=0.2),
        A.RandomCrop(width=200, height=200),
        A.PadIfNeeded(min_height=224, min_width=224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

def get_val_transform():
    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

# ----------------------------
# Custom Dataset Class
# ----------------------------
class FastDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        img, label = self.subset[idx]
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        return img, label

    def __len__(self):
        return len(self.subset)

# ----------------------------
# Model Training Loop
# ----------------------------
def train_model(train_loader, val_loader, model, optimizer, scheduler, scaler, loss_fn):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        with autocast():
            outputs = model(images).logits
            loss = loss_fn(outputs, labels)  # Use FocalLoss here

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            val_loss += loss_fn(outputs, labels).item()  # Use FocalLoss here
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

            all_preds.append(torch.softmax(outputs, dim=1).cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    # Calculate AUC
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    auc = roc_auc_score(all_labels, all_preds, multi_class='ovr')

    # Step the scheduler with validation loss
    scheduler.step(val_loss)

    return total_loss/len(train_loader), 100.0 * correct/total, val_loss/len(val_loader), 100.0 * val_correct/val_total, auc

# ----------------------------
# Cross-Validation Loop
# ----------------------------
def kfold_cross_validation(dataset):
    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=420)
    fold_aucs = []
    best_model = None  # Track best model

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold+1}/{K_FOLDS}...")

        # Split dataset
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Create data loaders
        train_dataset = FastDataset(train_subset, transform=get_train_transform())
        val_dataset = FastDataset(val_subset, transform=get_val_transform())

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

        # Initialize model, optimizer, and scheduler
        model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=len(dataset.classes), ignore_mismatched_sizes=True)
        model.to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        scaler = GradScaler()

        # Initialize FocalLoss
        loss_fn = FocalLoss(alpha=0.25, gamma=2.0, num_classes=len(dataset.classes))

        # Train model and evaluate AUC
        for epoch in range(EPOCHS):
            print(f"Epoch {epoch+1}/{EPOCHS}")
            train_loss, train_acc, val_loss, val_acc, auc = train_model(train_loader, val_loader, model, optimizer, scheduler, scaler, loss_fn)
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% | "
                  f"Val AUC: {auc:.4f}")

        fold_aucs.append(auc)
        if epoch == EPOCHS - 1:  # Save the last trained model
            best_model = model

    print(f"Average AUC across {K_FOLDS} folds: {np.mean(fold_aucs):.4f}")
    return best_model

# ----------------------------
# Main Function
# ----------------------------
def main():
    dataset = datasets.ImageFolder(root="/content/dataset/cropped")
    model = kfold_cross_validation(dataset)  # Get the trained model
    torch.save(model.state_dict(), "vit_model_augmented_crop.pth")
    print("✅ Training complete! Model saved as vit_model_augmented_crop.pth")

if __name__ == "__main__":
    main()

Fold 1/3...


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


<ipython-input-85-4568db1dbcad>:179: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-85-4568db1dbcad>:108: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.1289, Train Acc: 58.67% | Val Loss: 0.0573, Val Acc: 80.82% | Val AUC: 0.9571
Epoch 2/10


...

Train Loss: 0.0191, Train Acc: 93.23% | Val Loss: 0.0468, Val Acc: 87.59% | Val AUC: 0.9745
Epoch 9/10


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-85-4568db1dbcad>:108: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.0157, Train Acc: 94.51% | Val Loss: 0.0454, Val Acc: 88.21% | Val AUC: 0.9754
Epoch 10/10


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-85-4568db1dbcad>:108: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.0149, Train Acc: 94.87% | Val Loss: 0.0454, Val Acc: 88.41% | Val AUC: 0.9754
Average AUC across 3 folds: 0.9804
✅ Training complete! Model saved as vit_model_augmented_crop.pth


In [ ]:
# Re-initialize the dataset
from torchvision import datasets
from albumentations import Compose, Resize, Normalize
from albumentations.pytorch import ToTensorV2

dataset = datasets.ImageFolder(root="/content/dataset/relabeled_train")

# Load the trained model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(dataset.classes),
    ignore_mismatched_sizes=True
)
model.load_state_dict(torch.load("vit_model_augmented_crop.pth"))  # Load the saved model
model.to(device)

# ----------------------------
# 8) Test Inference
# ----------------------------
print("🔍 Running inference on test set...")

test_dir = "/content/dataset/test"
test_files = sorted(os.listdir(test_dir))

# Filter out non-image files (e.g., .DS_Store)
image_files = [file for file in test_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

# We'll reuse the clean transform for the test
test_transform = AlbumentationsTransformWrapper(no_augment())

test_results = []
model.eval()
with torch.no_grad():
    for file_name in image_files:
        file_path = os.path.join(test_dir, file_name)
        image = datasets.folder.default_loader(file_path)  # PIL loader
        image = test_transform(image).unsqueeze(0).to(device)

        outputs = model(image).logits
        probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()[0]
        test_results.append([file_name, *probs])

df = pd.DataFrame(test_results, columns=["id"] + dataset.classes)
df.to_csv("test_predictions_augmented_cropped.csv", index=False)
print("✅ Test predictions saved as test_predictions_augmented_cropped.csv")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔍 Running inference on test set...
✅ Test predictions saved as test_predictions_augmented_cropped.csv


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset, Subset
from transformers import ViTForImageClassification
from sklearn.metrics import roc_auc_score
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import random
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

# ----------------------------
# Configurations
# ----------------------------
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 5e-5
NUM_WORKERS = 4
PIN_MEMORY = True
K_FOLDS = 2

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

set_seed(420)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Optimized Augmentations
# ----------------------------
def get_train_transform():
    return A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.RandomRotate90(p=0.2),
        A.HueSaturationValue(p=0.2),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

def get_val_transform():
    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

# ----------------------------
# Custom Dataset Class
# ----------------------------
class FastDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        img, label = self.subset[idx]
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        return img, label

    def __len__(self):
        return len(self.subset)

# ----------------------------
# Model Training Loop
# ----------------------------
def train_model(train_loader, val_loader, model, optimizer, scaler):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        with autocast():
            outputs = model(images).logits
            loss = F.cross_entropy(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            val_loss += F.cross_entropy(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

            all_preds.append(torch.softmax(outputs, dim=1).cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    # Calculate AUC
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    auc = roc_auc_score(all_labels, all_preds, multi_class='ovr')

    return total_loss/len(train_loader), 100.0 * correct/total, val_loss/len(val_loader), 100.0 * val_correct/val_total, auc

# ----------------------------
# Cross-Validation Loop
# ----------------------------
def kfold_cross_validation(dataset):
    kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=420)
    fold_aucs = []
    best_model = None  # Track best model

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold+1}/{K_FOLDS}...")

        # Split dataset
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Create data loaders
        train_dataset = FastDataset(train_subset, transform=get_train_transform())
        val_dataset = FastDataset(val_subset, transform=get_val_transform())

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

        # Initialize model and optimizer
        model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=len(dataset.classes), ignore_mismatched_sizes=True)
        model.to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        scaler = GradScaler()

        # Train model and evaluate AUC
        for epoch in range(EPOCHS):
            print(f"Epoch {epoch+1}/{EPOCHS}")
            train_loss, train_acc, val_loss, val_acc, auc = train_model(train_loader, val_loader, model, optimizer, scaler)
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% | "
                  f"Val AUC: {auc:.4f}")

        fold_aucs.append(auc)
        if epoch == EPOCHS - 1:  # Save the last trained model
            best_model = model

    print(f"Average AUC across {K_FOLDS} folds: {np.mean(fold_aucs):.4f}")
    return best_model

# ----------------------------
# Main Function
# ----------------------------
def main():
    dataset = datasets.ImageFolder(root="/content/dataset/relabeled_train")
    model = kfold_cross_validation(dataset)  # Get the trained model
    torch.save(model.state_dict(), "vit_model_augmented.pth")
    print("✅ Training complete! Model saved as vit_model_augmented.pth")

if __name__ == "__main__":
    main()


Fold 1/2...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10


<ipython-input-87-41a7a3b78be2>:154: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-87-41a7a3b78be2>:87: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.8332, Train Acc: 64.89% | Val Loss: 0.4785, Val Acc: 82.25% | Val AUC: 0.9605
Epoch 2/10


...

Train Loss: 0.0911, Train Acc: 97.25% | Val Loss: 0.4367, Val Acc: 87.26% | Val AUC: 0.9758
Epoch 9/10


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-87-41a7a3b78be2>:87: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.0817, Train Acc: 97.49% | Val Loss: 0.4203, Val Acc: 88.28% | Val AUC: 0.9771
Epoch 10/10


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-87-41a7a3b78be2>:87: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.0609, Train Acc: 97.79% | Val Loss: 0.5650, Val Acc: 86.72% | Val AUC: 0.9736
Average AUC across 2 folds: 0.9748
✅ Training complete! Model saved as vit_model_augmented.pth


In [ ]:
#test uncrop model
# ----------------------------
# 8) Test Inference
# ----------------------------
print("🔍 Running inference on test set...")

# Load the trained model
model.load_state_dict(torch.load("/content/vit_model_augmented.pth"))
model.to(device)

test_dir = "/content/dataset/test"
test_files = sorted(os.listdir(test_dir))

# Filter out non-image files like .DS_Store
test_files = [f for f in test_files if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))]

# We'll reuse the clean transform for test
test_transform = AlbumentationsTransformWrapper(no_augment())

test_results = []
model.eval()
with torch.no_grad():
    for file_name in test_files:
        file_path = os.path.join(test_dir, file_name)
        image = datasets.folder.default_loader(file_path)  # PIL loader
        image = test_transform(image).unsqueeze(0).to(device)

        outputs = model(image).logits
        probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()[0]
        test_results.append([file_name, *probs])

# Save predictions
df = pd.DataFrame(test_results, columns=["id"] + dataset.classes)
df.to_csv("test_predictions_augmented.csv", index=False)

print("✅ Test predictions saved as test_predictions_augmented.csv")


🔍 Running inference on test set...
✅ Test predictions saved as test_predictions_augmented.csv


In [ ]:
#merge
import pandas as pd

# Load the base CSV (augmented cropped)
base_df = pd.read_csv("test_predictions_augmented_cropped.csv")

# Load the secondary CSV (test predictions)
secondary_df = pd.read_csv("test_predictions_augmented.csv")

# Merge without creating duplicates: fill missing values only
merged_df = base_df.set_index("id").combine_first(secondary_df.set_index("id")).reset_index()

# Save the merged result
merged_df.to_csv("test_predictions_merged.csv", index=False)

print("✅ Merging complete! Saved as test_predictions_merged.csv (No Duplicates)")

✅ Merging complete! Saved as test_predictions_merged.csv (No Duplicates)
